# Spectral Clustering
<hr />

&ensp;O _Spectral CLustering_, ou agrupamento espectral, é uma das classes de modelos preditivos, onde o processo de clusterização é baseado na teoria de grafos espectrais e em conceitos de álgebra linear. A teoria de grafos espectrais utiliza grafos para representar dados, onde a matriz de similaridade e a matriz laplaciana do grafo são fundamentais para capturar as relações entre os pontos. A matriz de similaridade indica quão próximos ou relacionados os pontos estão, enquanto a matriz laplaciana, que é derivada da similaridade, ajuda a entender a estrutura geral do grafo. Os valores e vetores próprios da matriz laplaciana, obtidos através da álgebra linear, permitem a redução da dimensionalidade dos dados, facilitando a projeção em um espaço onde os clusters são mais visíveis. 

### Importings e bibliotecas

In [1]:
import pandas as pd # pandas.
import numpy as np # numpy.
import matplotlib.pyplot as plt  # matplotlib.
import seaborn as sns  # seaborn.
from sklearn.decomposition import PCA # PCA.
from sklearn.cluster import SpectralClustering # spectral clustering.

### Leitura e visualização do DataFrame

In [3]:
df = pd.read_csv('data_updated.csv') 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100820 entries, 0 to 100819
Data columns (total 41 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   Apolice Sinistro                                 100820 non-null  int64  
 1   Codigo Empresa Sinistro                          100820 non-null  int64  
 2   Nome Empresa Sinistro                            100820 non-null  object 
 3   SEGURADO                                         100820 non-null  int64  
 4   Codigo Especialidade Sinistro                    100820 non-null  int64  
 5   Elegibilidade Sinistro                           100820 non-null  object 
 6   Sexo Sinistro                                    100820 non-null  object 
 7   Faixa-Etária Nova Sinistro                       100820 non-null  object 
 8   Descricao Plano Sinistro                         100820 non-null  object 
 9   Codigo Servico 

### Criação do modelo

&ensp;O _Spectral Clustering_ constrói uma matriz de afinidade que é de ordem quadrática em relação ao número de amostras. Com número grande de amostras, a memória necessária para criar essa matriz pode se tornar muito grande, resultando em erros de alocação de memória. Portanto, utilizamos o método PCA para reduzir a dimensionalidade, antes de clusterizar os dados.

In [4]:
numeric_columns = ["faixa-etaria_encoded_standardized", "valor-pago-sinistro_standardized", "quantidade_standardized", "doenca_relacionadas_encoded_standardized"] # features escolhidas.
dfNew = df[numeric_columns].sample(frac=0.2)
X = dfNew.values

# aplicado o PCA para reduzir a dimensionalidade e possibilidade a execução do modelo.
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

In [7]:
clustering = SpectralClustering(
    # hiperparâmetros com Tunning.
    n_clusters=4,
    n_neighbors=10,                
    affinity='nearest_neighbors', 
    n_init=1  
)
labels = clustering.fit_predict(X_pca)

c:\Users\Inteli\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.cluster import AgglomerativeClustering

# Definir função customizada para calcular e exibir as métricas
def custom_clustering_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    silhouette = silhouette_score(X, labels, metric='euclidean')
    davies_bouldin = davies_bouldin_score(X, labels)
    calinski_harabasz = calinski_harabasz_score(X, labels)
    
    # Exibir as métricas durante a busca
    print(f'Silhouette Score: {silhouette}')
    print(f'Davies-Bouldin Score: {davies_bouldin}')
    print(f'Calinski-Harabasz Score: {calinski_harabasz}')
    
    # Retornar o silhouette score como métrica para otimização
    return silhouette

# Definir parâmetros para busca
param_distributions = {
    'n_clusters': [2],
    'linkage': ['ward', 'complete', 'average', 'single'],
    'metric': ['euclidean', 'manhattan', 'cosine']
}

# Scorer customizado para Silhouette
silhouette_scorer = make_scorer(silhouette_score, metric='euclidean')

# Inicializar o RandomizedSearchCV com AgglomerativeClustering
agg_cluster = AgglomerativeClustering()

random_search = RandomizedSearchCV(
    estimator=agg_cluster,
    param_distributions=param_distributions,
    n_iter=3,  
    scoring=custom_clustering_scorer,  # Usando a função customizada para calcular as métricas
    cv=3,  
    verbose=1,
    n_jobs=-1
)

# Fazer o ajuste com os dados (X_pca é o conjunto de dados PCA transformado)
random_search.fit(X_pca)

# Resultados da busca
print("Melhores parâmetros: ", random_search.best_params_)

# Avaliar o modelo final nos melhores parâmetros encontrados
best_estimator = random_search.best_estimator_
labels = best_estimator.fit_predict(X_pca)

# Calcular e exibir as métricas para os melhores parâmetros
silhouette = silhouette_score(X_pca, labels, metric='euclidean')
davies_bouldin = davies_bouldin_score(X_pca, labels)
calinski_harabasz = calinski_harabasz_score(X_pca, labels)

print(f"Melhor Silhouette Score: {silhouette}")
print(f"Melhor Davies-Bouldin Score: {davies_bouldin}")
print(f"Melhor Calinski-Harabasz Score: {calinski_harabasz}")
